# 프로젝트 설명

In [1]:
import sys
sys.path.append("..")

In [2]:
from src.common import *
project_name = "dual_attention"
createfolder("./data")
createfolder("./model")
createfolder("./result")

# Data load

In [4]:
num_data = 14
data_list = []

for i in range(num_data):
    data_load = loadfile('./data/pre_data', f'Series {i+1}','csv')
    data_list.append(data_load)

csv file is loaded from ./data/pre_data/Series 1.csv
csv file is loaded from ./data/pre_data/Series 2.csv
csv file is loaded from ./data/pre_data/Series 3.csv
csv file is loaded from ./data/pre_data/Series 4.csv
csv file is loaded from ./data/pre_data/Series 5.csv
csv file is loaded from ./data/pre_data/Series 6.csv
csv file is loaded from ./data/pre_data/Series 7.csv
csv file is loaded from ./data/pre_data/Series 8.csv
csv file is loaded from ./data/pre_data/Series 9.csv
csv file is loaded from ./data/pre_data/Series 10.csv
csv file is loaded from ./data/pre_data/Series 11.csv
csv file is loaded from ./data/pre_data/Series 12.csv
csv file is loaded from ./data/pre_data/Series 13.csv
csv file is loaded from ./data/pre_data/Series 14.csv


## Data analysis

In [7]:
data_list[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584 entries, 0 to 583
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   FT-3061-2  584 non-null    float64
 1   FT-3061-3  584 non-null    float64
 2   FT-3061-4  584 non-null    float64
 3   LT-3061-2  584 non-null    float64
 4   FT-3062-1  584 non-null    float64
 5   TT-3061-3  584 non-null    float64
 6   TT-3061-4  584 non-null    float64
 7   PT-3061-3  584 non-null    float64
dtypes: float64(8)
memory usage: 36.6 KB


In [8]:
data_list[0].describe()

,FT-3061-2,FT-3061-3,FT-3061-4,LT-3061-2,FT-3062-1,TT-3061-3,TT-3061-4,PT-3061-3
count,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000
mean,49.701244,110.310137,51.842369,51.536209,37.705661,84.438432,115.694352,-0.955934
std,3.334363,1.935321,1.754774,4.888500,8.447288,1.020555,0.224212,0.001178
min,44.300000,106.224352,49.570106,43.190346,22.741410,82.759443,115.228622,-0.957960
25%,47.905582,109.243131,50.248903,47.889862,32.029866,83.575355,115.560835,-0.956745
50%,49.965846,110.110164,51.644214,49.491149,38.780335,84.288719,115.655523,-0.956250
75%,50.184730,110.911056,52.400047,56.072323,44.615609,85.339257,115.800546,-0.955155
max,57.313078,125.700000,55.734534,61.691086,55.465222,86.372862,116.357161,-0.952550


# Data preparation

In [5]:
from src import rnn

In [11]:
history_length = 60
future_length = 20
step = 1

history_var = ["FT-3061-2", "FT-3061-3", "FT-3061-4", "FT-3062-1", "TT-3061-3", "TT-3061-4", "LT-3061-2"]
future_var = ["TT-3061-3", "TT-3061-4", "LT-3061-2"]

history_series = []
future_series = []

for i in range(num_data):
    history, future = rnn.data2series(data_list[i], history_length, history_var, future_length, future_var,
                                      step, start_index=0, end_index=None)
    history_series.append(history)
    future_series.append(future)

In [12]:
history_series